In [4]:
import numpy as np
from numpy.linalg import svd as svd
from numpy.linalg import inv as inv

In [5]:
n = 50

def foo(i, j, k, l):
    return 1 / (i + j + k + l + 1)

A = np.fromfunction(foo, [n, n, n, n])
d = A.ndim

# Utils

In [7]:
def crop_vector(a: np.array, eps: float) -> int:
    """crop vector of positive values until all sum minus prefix sum < eps

    Args:
        a (np.array): vector of positive values
        eps (float): tolerance

    Returns:
        int: len of prefix sum
    """    
    r = 0
    cur_sum = 0
    sum = np.sum(a)

    while r < len(a):
        cur_sum += a[r]
        if sum - cur_sum < eps:
            return r + 1
        r += 1
    return r

def croped_svd(a: np.array, eps: float, full_matrices = False):
    """return cropped svd with absolute tolerance eps

    Args:
        a (np.array): matrix
        eps (float): absolute
        full_matrices (bool, optional): full_matrices for np.svd. Defaults to False.

    Returns:
        _type_: u, s, v - cropped svd
    """
    u, s, v = svd(a, full_matrices = full_matrices)
    r = crop_vector(s, eps)
    return u[:, :r], s[:r], v[:r, :]

# TT SVD

In [8]:
def TTSVD(A, eps): #from left to right
    d = A.ndim
    TT = A.reshape(A.shape[0], -1)
    u, s, v = croped_svd(TT, eps / (d**0.5))
    r1 = len(s)
    G1 = u
    TT = np.diag(s) @ v
    ranks = [r1]
    G = [G1]
    for k in range(1, d-1):
        TT = TT.reshape(ranks[k - 1] * A.shape[k], -1)
        u, s, v = svd(TT, full_matrices=False)
        ranks.append(crop_vector(s, eps / (d**0.5)))
        Gk = u[:, :ranks[k]].reshape(ranks[k-1], A.shape[k], ranks[k])
        G.append(Gk)
        TT = np.diag(s[:ranks[k]]) @ v[:ranks[k], :]
    G.append(TT)
    return G

In [9]:
G = TTSVD(A, 1e-8)
print([g.shape for g in G])

[(50, 13), (13, 50, 13), (13, 50, 13), (13, 50)]


In [10]:
def unfold_TT(G):
    d = len(G)
    g = G[0]
    for k in range(1, d-1):
        g = np.einsum(g, [i for i in range(g.ndim)], G[k], [g.ndim-1, g.ndim, g.ndim+1], [i for i in range(g.ndim - 1)] + [g.ndim, g.ndim+1]) # ...k,k...->...
    g = np.einsum(g, [i for i in range(g.ndim)], G[-1], [g.ndim - 1, g.ndim], [i for i in range(g.ndim -1)] + [g.ndim])
    return g

In [11]:
g = unfold_TT(G)
np.linalg.norm(g - A)

4.4758827043168295e-09

In [12]:
[np.linalg.norm(g)**2  for g in G], np.linalg.norm(g)**2

([12.999999999999996,
  12.999999999999986,
  12.999999999999986,
  952.4124701079983],
 952.4124701079974)

# Recompression

In [13]:
def recompess(G, eps, orthogonalization=False): #from right to left
    d = len(G)
    if orthogonalization:
        q, r = np.linalg.qr(G[0])
        G[0] = q
        np.einsum("ij,jkl->ikl", r, G[1])
        for k in range(1, d-1):
            q, r = np.linalg.qr(G[k].reshape(G[k-1].shape[-1] * G[k].shape[1], -1))
            G[k] = q.reshape(G[k-1].shape[-1], G[k].shape[1], G[k+1].shape[0])
            np.einsum("ij,jkl->ikl", r, G[k+1])

    u, s, v = croped_svd(G[d-1], eps / (d ** 0.5))
    Gnew = [v]
    W = u @ np.diag(s)
    for k in range(d-2, 0, -1):
        Gk = np.einsum("ijk,kl->ijl", G[k], W)
        u, s, v = croped_svd(Gk.reshape(Gk.shape[0], -1), eps / (d ** 0.5))
        W = u @ np.diag(s)
        Gnew.append(v.reshape(len(s), G[k].shape[1], Gnew[d-2-k].shape[0]))
    Gd = np.einsum("ij,jk->ik", G[0], W)
    u, s, v = croped_svd(Gd, eps / (d ** 0.5))
    Gnew.append(u @ np.diag(s) @ v)
    return Gnew[::-1]

In [16]:
Gnew = recompess(G[::-1], 1e-3, ortohonalization=)
print("old shape:", [g.shape for g in G], "\n" "new shape:", [g.shape for g in Gnew])

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (13,50,13)->(13,50,newaxis,13) (50,13)->(13,50) 

In [15]:
g1 = unfold_TT(G)
g2 = unfold_TT(Gnew)
np.linalg.norm(g1 - g2)

0.0003337647085426793

# TT cross


In [632]:
n = 50

def foo(i, j, k, l):
    return np.sin(i + j + k + l)

A = np.fromfunction(foo, [n, n, n, n])
d = A.ndim

In [633]:
def Sherman_Morrison_inv(A_inv, u, v):
    den = 1 + np.dot(v, A_inv @ u)
    return A_inv - np.outer(A_inv @ u,  v @ A_inv) / den

def MaxVol(A, eps=1e-2, maxit=20):
    m, r = A.shape


    for _try in range(20):
        set_i = np.random.choice(m, r, replace=False)
        cur_vol = abs(np.linalg.det(A[set_i, :]))
        if(cur_vol > 1e-5):
            break
    C = A[list(set_i), :]
    C_inv = np.linalg.inv(C)    
    for it in range(1, maxit):
        A_temp = np.abs(A @ C_inv)
        i, k = np.unravel_index(A_temp.argmax(), A_temp.shape)
        a_i = A[i, :]
        a_k = A[k, :]
        
        if A_temp[i, k] <= 1 + eps:
            break
        set_i[k] = i
        cur_vol *= A_temp[i, k]
        
        # C^-1 := (C - e_k(a_i - a_k))^-1
        C_inv = Sherman_Morrison_inv(C_inv, np.eye(1, r, k=k).ravel(), a_i - a_k)
    return np.sort(np.array(set_i))

def tensor_unfold(A: np.array, k : int) -> np.array:
    """unfold tensor by dimension k

    Args:
        A (np.aray): of size (n1, n2, ..., nd)
        k (int): dimension for unfold

    Returns:
        aray: matrix (nk, -1)
    """    
    return np.reshape(np.moveaxis(A, k, 0), (A.shape[k], -1))


In [634]:
def TTcross(A, ranks, eps = 1e-3, maxit=2, maxitcross = 100):
    d = A.ndim
    J = [np.random.choice(A.size // np.prod(A.shape[:k+1]), ranks[k], replace=False) for k in range(d-1)]
    I = [None for _ in range(d-1)]
    G = [None for _ in range(d)]
    for it in range(maxit):
        print(it)
        # forward iteration
        TT = A.reshape(A.shape[0], -1) 
        I[0] = MaxVol(TT[:, J[0]], eps, maxitcross)
        TT = TT[I[0], :] #  r1, n2*n3*...nd
        for k in range(1, d-1):
            TT = TT.reshape(ranks[k-1] * A.shape[k], -1) # r_{k-1}nk, n_{k+1}... n_d
            I[k] = MaxVol(TT[:, J[k]], eps, maxitcross)
            TT = TT[I[k], :]
        # backward iteration
        TT = tensor_unfold(A, d-1).T
        J[-1] = MaxVol(TT[I[-1], :].T, eps, maxitcross) # transpose for horizontal matrix
        TT = TT[:, J[-1]] # n1... n_d-1, r_d
        for k in range(len(J) - 2, -1, -1):
            TT = TT.reshape(-1, ranks[k + 1] * A.shape[k])
            J[k] = MaxVol(TT[I[k], :].T, eps, maxitcross) # transpose for horizontal matrix
            TT = TT[:, J[k]]
    
    # creating tensor train
    TT = A.reshape(A.shape[0], -1) 
    G[0] = TT[:, J[0]] @ inv(TT[I[0][:, None], J[0]])
    TT = TT[I[0], :] #  r1, n2*n3*...nd
    for k in range(1, d-1):
        TT = TT.reshape(ranks[k-1] * A.shape[k], -1) # r_{k-1}nk, n_{k+1}... n_d
        I[k] = MaxVol(TT[:, J[k]], eps, maxitcross)
        g = TT[:, J[k]] @ inv(TT[I[k][:, None], J[k]])
        G[k] = g.reshape(ranks[k-1], A.shape[k], ranks[k])
        TT = TT[I[k], :]
    G[-1] = TT
    return G

In [635]:
ranks = [2 for _ in range(A.ndim)]
G = TTcross(A, ranks, maxit=5)

0
1
2
3
4


In [636]:
g = unfold_TT(G)
np.linalg.norm(g - A)

6.9626029962047996e-12